# FatSecret API - Food Dataset Collection
## Project: Diet Plan App ML Dataset

**Tujuan**: Mengumpulkan data makanan dan nutrisi dari FatSecret API untuk training model machine learning

---


In [1]:
import requests
import json
import time
import pandas as pd
from datetime import datetime, timedelta
from google.colab import userdata
import logging

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("All libraries imported successfully!")


All libraries imported successfully!


In [2]:
# ============================================
# CONFIGURATION
# ============================================

CONFIG = {
    # API Credentials
    'CLIENT_ID': userdata.get('FATSECRET_CLIENT_ID'),
    'CLIENT_SECRET': userdata.get('FATSECRET_CLIENT_SECRET'),

    # ===================================
    # TARGETED SEARCH KEYWORDS
    # ===================================
    'KEYWORDS_EXCEL': 'only_food_list_with_en.xlsx',

    # ===================================
    # SEARCH & FILTERING PARAMETERS
    # ===================================
    'MAX_RESULTS_PER_PAGE': 20,
    'MAX_FOODS_PER_KEYWORD': 20,
    'DELAY_BETWEEN_REQUESTS': 0.5,   # seconds (rate limiting)

    # Filtering criteria
    'FOOD_TYPE_FILTER': 'Generic',   # Only Generic (not Brand)

    # ===================================
    # OUTPUT SETTINGS
    # ===================================
    'OUTPUT_FILE': 'fatsecret_raw_data_dump.xlsx',
}

# ============================================
# LOAD KEYWORDS FROM EXCEL
# ============================================

print("\n" + "="*70)
print("LOADING KEYWORDS FROM EXCEL")
print("="*70)

# Load Excel (kolom: food_name)
df_keywords = pd.read_excel(CONFIG['KEYWORDS_EXCEL'])

print(f"Loaded: {CONFIG['KEYWORDS_EXCEL']}")
print(f"   Total rows: {len(df_keywords)}")
print(f"   Columns: {list(df_keywords.columns)}")

# Ambil daftar keyword
keywords_list = df_keywords['food_name'].tolist()

# Bersihkan keyword kosong / NaN
keywords_list = [k for k in keywords_list if pd.notna(k) and str(k).strip() != '']

print(f"\nValid keywords to search: {len(keywords_list)}")
print(f"   Expected max foods: ~{len(keywords_list) * CONFIG['MAX_FOODS_PER_KEYWORD']}")
print(f"   Estimated time: ~{len(keywords_list) * CONFIG['DELAY_BETWEEN_REQUESTS'] / 60:.1f} minutes")

print("\n Sample keywords:")
for i, kw in enumerate(keywords_list[:10], 1):
    print(f"   {i}. {kw}")

print("="*70)



LOADING KEYWORDS FROM EXCEL
Loaded: only_food_list_with_en.xlsx
   Total rows: 70
   Columns: ['food_name']

Valid keywords to search: 70
   Expected max foods: ~1400
   Estimated time: ~0.6 minutes

 Sample keywords:
   1. Egg
   2. Coffee
   3. Tea
   4. Soup
   5. Ice Cream
   6. Pudding
   7. Nuggets
   8. Curry
   9. Potatoes
   10. Noodles


In [4]:
# ============================================
# AUTHENTICATION
# ============================================

class FatSecretAuth:
    """Handle FatSecret OAuth 2.0 authentication"""

    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
        self.access_token = None
        self.token_expiry = None

    def get_access_token(self):
        """Get new access token"""
        token_url = 'https://oauth.fatsecret.com/connect/token'
        auth_data = {
            'grant_type': 'client_credentials',
            'scope': 'basic'
        }

        logger.info("🔑 Requesting access token...")

        try:
            response = requests.post(
                token_url,
                auth=(self.client_id, self.client_secret),
                data=auth_data,
                timeout=10
            )

            if response.status_code == 200:
                data = response.json()
                self.access_token = data['access_token']
                self.token_expiry = time.time() + data.get('expires_in', 3600)
                logger.info("Access token obtained!")
                return self.access_token
            else:
                logger.error(f"Token request failed: {response.text}")
                return None
        except Exception as e:
            logger.error(f"Authentication error: {str(e)}")
            return None

    def get_valid_token(self):
        """Get valid token (refresh if needed)"""
        if self.access_token is None or self.is_token_expired():
            return self.get_access_token()
        return self.access_token

    def is_token_expired(self):
        """Check if token expired"""
        if self.token_expiry is None:
            return True
        return time.time() >= (self.token_expiry - 300)

# Initialize
print("\n" + "="*70)
print("AUTHENTICATION")
print("="*70)

auth = FatSecretAuth(CONFIG['CLIENT_ID'], CONFIG['CLIENT_SECRET'])
ACCESS_TOKEN = auth.get_access_token()

if ACCESS_TOKEN:
    print(f"Authenticated successfully!")
else:
    print("Authentication failed! Check credentials.")



AUTHENTICATION
Authenticated successfully!


In [5]:
# ============================================
# API FUNCTIONS
# ============================================

def search_foods_v1(query, token, max_results=20, page_number=0):
    """
    Search foods using FatSecret API
    Returns raw API response
    """
    max_results = min(max(1, max_results), 50)

    headers = {'Authorization': f'Bearer {token}'}
    url = "https://platform.fatsecret.com/rest/foods/search/v1"

    params = {
        'search_expression': query,
        'format': 'json',
        'max_results': max_results,
        'page_number': page_number
    }

    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)

        if response.status_code == 200:
            data = response.json()
            if 'error' in data:
                logger.error(f"API Error: {data['error']}")
                return None
            return data
        else:
            logger.error(f"HTTP Error {response.status_code}")
            return None

    except Exception as e:
        logger.error(f"Error: {str(e)}")
        return None

print("API functions loaded")


API functions loaded


In [6]:
# ============================================
# COLLECTION FUNCTION (RAW DATA ONLY)
# ============================================

def collect_raw_foods_for_keyword(query, token, max_per_page=20, max_foods=10):
    """
    Collect RAW food data (NO preprocessing, NO name prioritization)

    - Hanya ambil food_type == 'Generic'
    - Ambil sampai max_foods hasil pertama dari API (API sudah urutkan by relevance)
    - Return list of dict:
      - food_id
      - food_name
      - food_type
      - description
      - search_keyword
    """
    logger.info(f"🔍 Searching '{query}'")

    # Search page 0 only
    results = search_foods_v1(query, token, max_per_page, page_number=0)

    if not results or 'foods' not in results:
        logger.warning("   No results")
        return []

    foods_data = results['foods']
    if 'food' not in foods_data:
        logger.warning("   No food data")
        return []

    # Parse foods
    foods = foods_data['food']
    if isinstance(foods, dict):
        foods = [foods]

    # Filter: Generic only
    generic_foods = [f for f in foods if f.get('food_type') == CONFIG['FOOD_TYPE_FILTER']]

    if not generic_foods:
        logger.warning("   No Generic foods")
        return []

    selected_foods = generic_foods[:max_foods]

    raw_foods = []

    for food in selected_foods:
        food_name = food.get('food_name', 'Unknown')
        description = food.get('food_description', '')

        raw_food = {
            'food_id': food.get('food_id'),
            'food_name': food_name,
            'food_type': food.get('food_type', 'Unknown'),
            'description': description,
            'search_keyword': query,
        }

        raw_foods.append(raw_food)
        logger.info(f"   {food_name}")

    logger.info(f"   Collected {len(raw_foods)} foods for '{query}'")

    return raw_foods

print("Collection function ready (RAW, Generic-only, no prioritization)")


Collection function ready (RAW, Generic-only, no prioritization)


In [9]:
# ============================================
# EXECUTE COLLECTION
# ============================================

all_raw_data = []
stats = {
    'total_keywords': 0,
    'successful': 0,
    'failed': 0,
}

print("\n" + "="*70)
print("STARTING RAW DATA COLLECTION")
print("="*70)

for idx, keyword in enumerate(keywords_list, 1):
    stats['total_keywords'] += 1

    # Get fresh token
    token = auth.get_valid_token()

    # Collect RAW data
    foods = collect_raw_foods_for_keyword(
        query=keyword,
        token=token,
        max_per_page=CONFIG['MAX_RESULTS_PER_PAGE'],
        max_foods=CONFIG['MAX_FOODS_PER_KEYWORD']
    )

    if foods:
        all_raw_data.extend(foods)
        stats['successful'] += 1
    else:
        stats['failed'] += 1
        print(f"   {keyword}: No data")

    # Progress log setiap 10 keyword
    if idx % 10 == 0:
        print(f"\n--- Progress: {idx}/{len(keywords_list)} keywords processed ---")

    # Rate limiting
    time.sleep(CONFIG['DELAY_BETWEEN_REQUESTS'])

# ============================================
# DEDUPLICATION
# ============================================

print(f"\n{'='*70}")
print("🧹 REMOVING DUPLICATES")
print(f"{'='*70}")

df_temp = pd.DataFrame(all_raw_data)
print(f"Before deduplication: {len(df_temp)} rows")

# Cek duplicate food_id
duplicates = df_temp[df_temp.duplicated(subset=['food_id'], keep='first')]
if len(duplicates) > 0:
    print(f"\n Found {len(duplicates)} duplicate food_ids (showing up to 10):")
    for _, dup in duplicates.head(10).iterrows():
        print(f"   • ID {dup['food_id']:>6}: {dup['food_name']} (keyword: {dup['search_keyword']})")

# Drop duplicates
df_temp = df_temp.drop_duplicates(subset=['food_id'], keep='first')

print(f"\nAfter deduplication: {len(df_temp)} rows")
print(f"Removed: {len(all_raw_data) - len(df_temp)} duplicate rows")

# Convert back to list of dict
all_raw_data = df_temp.to_dict('records')

print(f"\n{'='*70}")
print("COLLECTION COMPLETE!")
print(f"{'='*70}")
print("Statistics:")
print(f"   • Keywords searched : {stats['total_keywords']}")
print(f"   • Successful        : {stats['successful']}")
print(f"   • Failed            : {stats['failed']}")
print(f"   • Total foods final : {len(all_raw_data)}")
print("="*70)



STARTING RAW DATA COLLECTION

--- Progress: 10/70 keywords processed ---

--- Progress: 20/70 keywords processed ---

--- Progress: 30/70 keywords processed ---

--- Progress: 40/70 keywords processed ---

--- Progress: 50/70 keywords processed ---

--- Progress: 60/70 keywords processed ---

--- Progress: 70/70 keywords processed ---

🧹 REMOVING DUPLICATES
Before deduplication: 563 rows

 Found 16 duplicate food_ids (showing up to 10):
   • ID   5279: Tangerine (keyword: Tangerine)
   • ID  35890: Tangerines (Mandarin Oranges) (keyword: Tangerine)
   • ID   6229: Green String Beans (keyword: Green beans)
   • ID   6314: Cooked Green String Beans (from Fresh) (keyword: Green beans)
   • ID   6370: Cooked Green String Beans (Fat Added in Cooking) (keyword: Green beans)
   • ID   6397: Cooked Green String Beans (from Frozen) (keyword: Green beans)
   • ID   6346: Cooked Green String Beans (from Fresh, Fat Added in Cooking) (keyword: Green beans)
   • ID   6462: Cooked Green String Beans

In [ ]:
# ============================================
# SAVE RAW DATA
# ============================================

print("\n" + "="*70)
print("SAVING RAW DATA")
print("="*70)

df_raw = pd.DataFrame(all_raw_data)

print("\n Dataset Info:")
print(f"   Shape  : {df_raw.shape}")
print(f"   Columns: {list(df_raw.columns)}")

print("\n Sample data (first 5 rows):")
print(df_raw[['food_name', 'food_type', 'search_keyword']].head(5))

# Save
if CONFIG['OUTPUT_FILE'].lower().endswith('.csv'):
    df_raw.to_csv(CONFIG['OUTPUT_FILE'], index=False)
else:
    df_raw.to_excel(CONFIG['OUTPUT_FILE'], index=False)

print(f"\n Raw data saved to: {CONFIG['OUTPUT_FILE']}")
print(f"   Total rows   : {len(df_raw)}")
print(f"   Total columns: {len(df_raw.columns)}")

print("\n" + "="*70)
print("COLLECTION PHASE COMPLETE!")
print("="*70)



💾 SAVING RAW DATA

📊 Dataset Info:
   Shape  : (547, 5)
   Columns: ['food_id', 'food_name', 'food_type', 'description', 'search_keyword']

📄 Sample data (first 5 rows):
                       food_name food_type search_keyword
0                            Egg   Generic            Egg
1                      Fried Egg   Generic            Egg
2                     Boiled Egg   Generic            Egg
3  Scrambled Egg (Whole, Cooked)   Generic            Egg
4                      Egg White   Generic            Egg

✅ Raw data saved to: fatsecret_raw_data_dump.csv
   Total rows   : 547
   Total columns: 5

🎯 COLLECTION PHASE COMPLETE!
